In [ ]:
# Carreguem les dades
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
data = pd.read_csv("dades.csv", decimal = ',')

nFiltres =3
nf = nFiltres+1
filtres = {}

for i in range(1,nf):
    filtres["filtre{}".format(i)] = pd.DataFrame()
    OV = data["OV{}".format(i)]
    AP = data["AP{}".format(i)]
    Q = data["Q{}".format(i)]
    filtres["filtre{}".format(i)]["OV"] = OV
    filtres["filtre{}".format(i)]["AP"] = AP
    filtres["filtre{}".format(i)]["Q"] = Q

# Creem una variable "estat" on definirem l'estat del filtre
for filtre in filtres:
    filtres[filtre]["Estat"] = pd.Series()
    filtres[filtre] = filtres[filtre].reset_index()

# Es defineix de forma molt simple si els filtres estàn oberts o tancats. Lo dels filtres 
# reiniciats no ho acabo d'entendre
OVmin=20
for filtre in filtres:
    for m in range(0,len(filtres[filtre]["OV"])):
        if filtres[filtre]["OV"][m] > OVmin:
            filtres[filtre].loc[m,"Estat"] = "Actiu"
        else:
            filtres[filtre].loc[m,"Estat"] = "Tancat"
# Creo variable Vlum acumulat 
for filtre in filtres:
    filtres[filtre]["Vac"] = pd.Series()
    
# Omplenem les dades

for filtre in filtres:
    for m in range(0,len(filtres[filtre]["Vac"])):
        if filtres[filtre]["Estat"][m] == "Tancat":
            filtres[filtre].loc[m, "Vac"]= 0

# Si el filtre està obert:
    # CAL CAMBIAR LES UNITATS!!!
for filtre in filtres:
    for m in range(0,len(filtres[filtre]["Vac"])):
        if filtres[filtre]["Estat"][m] == "Actiu" and m > 0:
            filtres[filtre].loc[m, "Vac"] = \
            filtres[filtre]["Q"][m] + \
            filtres[filtre]["Vac".format(i)][m-1]


for filtre in filtres:
    if len(filtres[filtre]["Q"])-filtres["filtre1"]["Q"].count() > 1:
        print("ERROR FATAL: HI HA MASSES NANS EN ALGUN FILTRE!")
        
# Altres variables que cal crear:
# DESPRÉS D'ELIMINAR NANS
    # PERUA.CARREGA/CABAL --> error extrany
    # PERDUA CARREGA/ VOLUM FILTRAT --> error extrany

    # TERBOLESA ACUMULADA: faig una variable per a cada cicle de cada filtre? o com??
        #està fet d'aquest mode.
    # CleanBeadHeadLoss: Em falta saber què és 
        # https://www.google.com/url?\
        # sa=t&rct=j&q=&esrc=s&source=web&cd=16&ved=2ah\
        # UKEwjy87O0p97mAhWS0eAKHYGyCYsQFjAPegQIBhAC&url=ht\ 
        # tp%3A%2F%2Fmimoza.marmara.edu.tr%2F~bilge.alpaslan%2Fe\
        # nve301%2FLectures%2FChp_12.pdf&usg=AOvVaw2BtrE16Fa_RsUm5or11MpF
# HORES TOTALS FILTRADES [HTF]
# Creo variable Vlum acumulat 
Temps_interval = 1

for filtre in filtres:
    filtres[filtre]["HTF"] = pd.Series()


for filtre in filtres:
    for i in range(0,len(filtres[filtre]["HTF"])):
        if filtres[filtre]["Estat"][i] == "Tancat":
            filtres[filtre].loc[i, "HTF"]=0

for filtre in filtres:
    for m in range(0,len(filtres[filtre]["HTF"])):
        if filtres[filtre]["Estat"][m] == "Actiu" and m > 0:
            filtres[filtre].loc[m, "HTF"] = filtres[filtre]["HTF"][m-1] + Temps_interval
# Treiem els NaNs
for i in range(1,nf):
    filtres["filtre{}".format(i)] = filtres["filtre{}".format(i)].reset_index().\
    dropna().set_index("index")
# L'he liat parda amb els indexs: aquest codi m'els recuperararà

for filtre in filtres:
    filtres[filtre] = filtres[filtre].reset_index()
# Pèrdua de carrega / Cabal [AP.Q]
# PROBLEMA: QUAN HE TRET ELS NNANS HE BORRAT ELS INDEX. CAL RECUPERAR-LOS.
for filtre in filtres:
    filtres[filtre]["AP:Q"] = pd.Series()
    
for filtre in filtres:
    for m in range(0,len(filtres[filtre]["AP:Q"])):
        filtres[filtre].loc[m, "AP:Q"] = filtres[filtre]["AP"][m] / filtres[filtre]["Q"][m]

# SURT COM UN PSEUDOERROR  DESPRÉS D'EXECUTAR AQUESTA PORCIÓ DE CODI. NO SE SI CAL TENIR-HO 
# EN COMPTE, JA QUE SEMBLA QUE EM TREU ELS RESULTATS IGUALMENT. TOT I AIXÍ, CALDRIA FER-LI
# UNA ULLADA
    
# Pèrdua de carrega / volum filtrat [AP:VF]

for filtre in filtres:
    filtres[filtre]["AP:Vac"] = pd.Series()
for filtre in filtres:
    for m in range(0, len(filtres[filtre]["AP:Vac"])):
        filtres[filtre].loc[m, "AP:Vac"] = filtres[filtre]["AP"][m] / \
        filtres[filtre]["Vac"][m]
    
# Terbolesa acumulada: suposant que es reinicia per a cada filtre
Terb = data["Terb_DL"]

for filtre in filtres:
    filtres[filtre]["TerbAc"] = pd.Series()
for filtre in filtres:
    for m in range(0, len(filtres[filtre]["TerbAc"])):
        if filtres[filtre]["Estat"][m] == "Tancat":
            filtres[filtre].loc[m, "TerbAc"] = Terb[m]

# TERBOLESA ACUMULADA PART 2 AQUI HI HA PROBLEMES:            
for filtre in filtres:
    for m in range(0, len(filtres[filtre]["TerbAc"])):
        if filtres[filtre]["Estat"][m] == "Actiu" and m > 0:
            filtres[filtre].loc[m, "TerbAc"] = filtres[filtre]["TerbAc"][m-1]\
            + Terb[m]
# Informació rellevant:
    # Com els decimals de "Terb" al csv eren ",", he canviat com es llegeixen les dades al 
    # python. Ara sembla que em fa les operacions pertinents, però a Terb tenim un fotiment
    # de NANS. Caldrà fer alguna cosa. 
    
# D'altra banda, cal recordar que encara no he solucionat els problemes amb els NaNs que 
# pot haverhi en "Q" i demés !!! <<--- -IMPORTANT SOLUCIONAR-HO

In [ ]:
### FINS AQUÍ SEMBLA QUE TINDRIA LES DADES "PREPARADES", PERO FALTA RECUPERAR O ESTIMAR
# VALORS PERDUTS (NANS) I RECUPERAR TOTS ELS VALORS PERDUTS DESPRÉS D'APLICAR ELS NANS. 

In [28]:
# ANÀLISIS ESTADÍSTICS # INTENT 1

In [21]:
BSSfiltres = filtres["filtre1"]

In [22]:
# HAIG D'ELIMINAR LES FILES ON L'ESTAT DEL FILTRE SIGUI TANCAT
for i in range(0, len(BSSfiltres["Estat"])):
    if BSSfiltres["Estat"][i] == "Tancat":
        BSSfiltres = BSSfiltres.drop(i)


In [27]:
BSSfiltres

,index,level_0,OV,AP,Q,Estat,Vac,HTF,AP:Q,AP:Vac,TerbAc
0,1,1,65.0,1233,134,Actiu,134.0,1.0,9.201493,9.201493,NaN
1,2,2,69.0,1359,145,Actiu,279.0,2.0,9.372414,4.870968,NaN
2,3,3,72.0,1478,149,Actiu,428.0,3.0,9.919463,3.453271,NaN
4,5,5,55.0,119,143,Actiu,143.0,1.0,0.832168,0.832168,NaN
5,6,6,57.0,159,148,Actiu,291.0,2.0,1.074324,0.546392,NaN
...,...,...,...,...,...,...,...,...,...,...,...
3211,3212,3212,55.0,1095,103,Actiu,2771.0,25.0,10.631068,0.395164,34.93
3212,3213,3213,53.0,1138,96,Actiu,2867.0,26.0,11.854167,0.396931,36.65
3213,3214,3214,51.0,1136,89,Actiu,2956.0,27.0,12.764045,0.384303,38.24
3214,3215,3215,54.0,1243,97,Actiu,3053.0,28.0,12.814433,0.407141,39.71


In [9]:
# Treiem els NaNs
BSSfiltres

,index,level_0,OV,AP,Q,Estat,Vac,HTF,AP:Q,AP:Vac,TerbAc
0,1,1,65.0,1233,134,Actiu,134.0,1.0,9.201493,9.201493,NaN
1,2,2,69.0,1359,145,Actiu,279.0,2.0,9.372414,4.870968,NaN
2,3,3,72.0,1478,149,Actiu,428.0,3.0,9.919463,3.453271,NaN
4,5,5,55.0,119,143,Actiu,143.0,1.0,0.832168,0.832168,NaN
5,6,6,57.0,159,148,Actiu,291.0,2.0,1.074324,0.546392,NaN
...,...,...,...,...,...,...,...,...,...,...,...
3211,3212,3212,55.0,1095,103,Actiu,2771.0,25.0,10.631068,0.395164,34.93
3212,3213,3213,53.0,1138,96,Actiu,2867.0,26.0,11.854167,0.396931,36.65
3213,3214,3214,51.0,1136,89,Actiu,2956.0,27.0,12.764045,0.384303,38.24
3214,3215,3215,54.0,1243,97,Actiu,3053.0,28.0,12.814433,0.407141,39.71
